### prueba scrapear!

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
import os
from PIL import Image 

In [23]:
header = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) "
          "Chrome/81.0.4044.141 Safari/537.36"}

#### carrefour

In [24]:
list_pages_carrefour = list(range(0,336,24))
table_pages_carrefour=[]
for number in list_pages_carrefour:
    url_carrefour = f'https://www.carrefour.es/supermercado/bebidas/cerveza/todas-las-cervezas/N-1uq8b5u/c?No={number}'
    html = requests.get(url_carrefour, headers=header).text
    soup_carrefour = BeautifulSoup(html, 'lxml')
    table_carrefour = soup_carrefour.find_all({'article':'product-card-item'})
    table_pages_carrefour.append(table_carrefour)

In [4]:
table_pages_carrefour[0][23]

<article class="product-card-item">
<div class="right-side">
<div class="bg-promocion-copy">
<p class="promocion-copy">Cupón 50%</p>
</div>
<div class="photo">
<a class="js-gap-product-click-super" data-product-discount="" data-productid="526517627" data-special-campaign="qvpromo750001_cupon-50" href="/supermercado/cerveza-san-miguel-especial-lager-pack-de-28-latas-de-33-cl-san-miguel/R-526517627/p" title="Cerveza San Miguel especial Lager pack de 28 latas de 33 cl.">
<img alt="Cerveza San Miguel especial Lager pack de 28 latas de 33 cl." aria-describedby="product-526517627" sizes="(max-width: 280px) 80px,(max-width: 490px) 100px, (max-width: 768px) 560px, (max-width: 991px) 280px, (max-width: 1280px) 560px" src="https://static.carrefour.es/hd_280x_/img_pim_food/718561_00_1.jpg" srcset="https://static.carrefour.es/hd_560x_/img_pim_food/718561_00_1.jpg 560w, https://static.carrefour.es/hd_280x_/img_pim_food/718561_00_1.jpg 280w, https://static.carrefour.es/hd_100x_/img_pim_food/718561_0

In [5]:
len(table_pages_carrefour[0])

25

In [6]:
data_raw_carrefour=[]
for i in range(len(table_pages_carrefour)):
    for j in range(len(table_pages_carrefour[i])):
        rows_carrefour = table_pages_carrefour[i][j].find_all(['span','p','a'], {'class': ['price','price-less','format-price','js-gap-product-click-super']})
        promotion_carrefour = table_pages_carrefour[i][j].find_all(['p'], {'class': ['promocion-copy']})
        if len(rows_carrefour) != 0:
            if len(promotion_carrefour) != 0:
                data_raw_carrefour.append(rows_carrefour + promotion_carrefour)
            else:
                data_raw_carrefour.append(rows_carrefour)
                
            

In [7]:
data_raw_carrefour[20]

[<a class="js-gap-product-click-super" data-product-discount="" data-productid="521028259" data-special-campaign="" href="/supermercado/cerveza-mahou-5-estrellas-especial-pack-de-12-botellas-de-25-cl-mahou/R-521028259/p" title="Cerveza Mahou 5 Estrellas especial pack de 12 botellas de 25 cl.">
 <img alt="Cerveza Mahou 5 Estrellas especial pack de 12 botellas de 25 cl." aria-describedby="product-521028259" sizes="(max-width: 280px) 80px,(max-width: 490px) 100px, (max-width: 768px) 560px, (max-width: 991px) 280px, (max-width: 1280px) 560px" src="https://static.carrefour.es/hd_280x_/img_pim_food/523315_00_1.jpg" srcset="https://static.carrefour.es/hd_560x_/img_pim_food/523315_00_1.jpg 560w, https://static.carrefour.es/hd_280x_/img_pim_food/523315_00_1.jpg 280w, https://static.carrefour.es/hd_100x_/img_pim_food/523315_00_1.jpg 100w, https://static.carrefour.es/hd_100x_/img_pim_food/523315_00_1.jpg  80w" width="100%"/>
 </a>,
 <span class="price">5,95 €</span>,
 <p class="format-price">
  P

In [8]:
len(data_raw_carrefour)

336

In [9]:
#price in euros
# price_raw = data_raw[0][1].text

def get_price(data_text):
    price_raw = data_text.text
    price = re.sub('\xa0€','',price_raw)
    price = re.sub('\n','',price)
    return float(re.sub(",",'.',price))

In [10]:
#price per liter
# price_l_raw = data_raw[0][2].text

def get_price_l(data_text):
    price_l_raw = data_text.text
    price_l = re.findall('[0-9]+,[0-9]+', price_l_raw)
    price_liter = re.sub(',','.',price_l[0])
    return float(price_liter)

In [11]:
#title
# title_raw = data_raw[0][3].text

def get_title(data_text):
    title_raw = data_text.text
    title = re.sub("\n",'',title_raw)
    return title

In [12]:
#promotion product

def get_promotion(data_text):
        promotion_raw = data_text.text
        promotion = re.sub("\n",'',promotion_raw)
        return promotion


In [13]:
#brand
def get_brand(data_text):
    brand_raw = data_text.text
    brand = re.split('(Cerveza )|( pack)|( botella)|( lata)+|(cl)',brand_raw)
    return brand[6].lower()

In [14]:
#container (carrefour  / alcampo)
def get_container(data_text):
    container_raw = data_text.text
    container = re.findall('botella|lata|barril',container_raw)
    if len(container)!=0:
        return container[0]
    else:
        return 'No specified'

In [15]:
#volumen
def get_volumen_unid(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9]+[ mcl]+',volumen_raw)
        return volumen_l[0]

In [16]:
#quantity (carrefour  / alcampo)
def get_quantity_pack(data_text):
    quantity_raw = data_text.text
    quantity_split = re.split('pack de|pack',quantity_raw)
    if len(quantity_split)==2:
        quantity = re.findall('[0-9]+',quantity_split[1])
        return quantity[0]
    else:
        return 1

In [17]:
#image
def get_image(data_text):
    return data_text.find_all('img')[0].get('src')

In [25]:
data_raw_carrefour[1][3]

<a class="js-gap-product-click-super" data-product-discount="0,52" data-special-campaign="promof2340005_2a-al-50" href="/supermercado/cerveza-cruzcampo-pilsen-pack-de-28-latas-de-33-cl-cruzcampo/R-562901511/p" title="Ir a Cerveza Cruzcampo Pilsen pack de 28 latas de 33 cl.">
<p class="title-product">
Cerveza Cruzcampo Pilsen pack de 28 latas de 33 cl.</p>
</a>

In [18]:
data_beer_carrefour = pd.DataFrame(index=range(0,len(data_raw_carrefour)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_carrefour)):
    data_beer_carrefour.iloc[beer_number,0] = get_price(data_raw_carrefour[beer_number][1])
    data_beer_carrefour.iloc[beer_number,1] = get_price_l(data_raw_carrefour[beer_number][2])
    data_beer_carrefour.iloc[beer_number,2] = get_title(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,4] = get_brand(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,5] = get_container(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,6] = get_volumen_unid(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,7] = get_quantity_pack(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,8] = get_image(data_raw_carrefour[beer_number][0])
    data_beer_carrefour.iloc[beer_number,9] = 'Carrefour'
    if len(data_raw_carrefour[beer_number]) > 4:
        data_beer_carrefour.iloc[beer_number,3] = get_promotion(data_raw_carrefour[beer_number][4])
    else:
        data_beer_carrefour.iloc[beer_number,3] = 'No promotion'


In [19]:
data_beer_carrefour.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,8.99,1.51,Cerveza Heineken Lager pack de 18 latas de 33 cl.,No promotion,heineken lager,lata,33 cl,18,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
1,14.28,1.55,Cerveza Cruzcampo Pilsen pack de 28 latas de 3...,2ª al -50%,cruzcampo pilsen,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
2,5.15,1.56,Cerveza El Aguila pack de 10 latas de 33 cl.,No promotion,el aguila,lata,33 cl,10,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
3,0.25,0.76,Cerveza Carrefour Holandesa lata 33 cl.,No promotion,carrefour holandesa,lata,33 cl,1,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
4,10.64,1.15,Cerveza Mahou Clásica pack de 28 latas de 33 cl.,No promotion,mahou clásica,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour


In [20]:
sum(data_beer_carrefour['price'])

926.7800000000002

## Alcampo

In [3]:
list_pages_alcampo = list(range(0,10))
table_pages_alcampo=[]
for number in list_pages_alcampo:
    url_alcampo = f'https://www.alcampo.es/compra-online/bebidas/cervezas/c/W1107?q=%3Arelevance&page={number}'
    html = requests.get(url_alcampo, headers=header).text
    soup_alcampo = BeautifulSoup(html, 'lxml')
    table_alcampo = soup_alcampo.find_all('div',{'class':'productGridItem'})
    table_pages_alcampo.append(table_alcampo)
    

In [4]:
len(table_pages_alcampo[0])

48

In [5]:
len(table_pages_alcampo)

10

In [6]:
data_raw_alcampo=[]
for i in range(len(table_pages_alcampo)):
    for j in range(len(table_pages_alcampo[i])):
        rows_alcampo = table_pages_alcampo[i][j].find_all(['span','div'],{'class': ['price',
                                                                                    'productName',
                                                                                    'thumb cut-alt-img']})
        promotion_alcampo = table_pages_alcampo[i][j].find_all('div', {'class': 'financiacionMensual'})
        if len(rows_alcampo) != 0:
            if len(promotion_alcampo) != 0:
                data_raw_alcampo.append(rows_alcampo + promotion_alcampo)
            else:
                data_raw_alcampo.append(rows_alcampo)
                

In [7]:
len(data_raw_alcampo)

440

In [8]:
#price alcampo / corte ingles / Dia
def get_price_alcampo(data_text):
    price_raw = data_text.text
    price = re.findall('[0-9,]+',price_raw)
    return float(re.sub(",",'.',price[0]))


In [9]:
#price_liter alcampo / corte ingles / Dia
def get_price_liter_alcampo(data_text):
    price_raw = data_text.text
    price = re.findall('[0-9,]+',price_raw)
    if len(price)==1:
        return float(re.sub(",",'.',price[0]))
    else:
        return float(re.sub(",",'.',price[1]))


In [10]:
#title alcampo / corte ingles
def get_tittle_alcampo(data_text):
    tittle_raw = data_text.text
    tittle = re.sub('\n','',tittle_raw)
    return tittle.lower()


In [11]:
#promotion alcampo
def get_promotion_alcampo(data_text):
    promotion_raw = data_text.text
    promotion = re.sub('\r\n\r\n\t\t\t\t\t\t\t\t\t\t','',promotion_raw)
    return promotion

In [12]:
#brand alcampo
def get_brand_alcampo(data_text):
    brand_raw = data_text.text
    brand = re.sub('\n','',brand_raw)
    brand = brand.lower()
    brand_2 = re.sub('cervez\S+','',brand)
    return brand_2

In [13]:
#volumen alcampo
def get_volumen_unid_alcampo(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9.,]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9.,]+[ mclL]+',volumen_raw)
        if len(volumen_l)==2:
            return volumen_l[1]
        else:
            return volumen_l[0]

In [14]:
#image alcampo
def get_image_alcampo(data_text):
    return data_text.find_all('img')[0].get('data-blzsrc')

In [103]:
data_raw_alcampo[169][0].find_all('img')[0].get('data-blzsrc')

'https://www.alcampo.es/media/h48/h2d/9558660513822.gif'

In [26]:
data_beer_alcampo= pd.DataFrame(index=range(0,len(data_raw_alcampo)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_alcampo)):
    data_beer_alcampo.iloc[beer_number,0] = get_price_alcampo(data_raw_alcampo[beer_number][2])
    data_beer_alcampo.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_alcampo[beer_number][2])
    data_beer_alcampo.iloc[beer_number,2] = get_tittle_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,4] = get_brand_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,5] = get_container(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,7] = get_quantity_pack(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,8] = get_image_alcampo(data_raw_alcampo[beer_number][0])
    data_beer_alcampo.iloc[beer_number,9] = 'Alcampo'
    if len(data_raw_alcampo[beer_number]) > 3:
        data_beer_alcampo.iloc[beer_number,3] = get_promotion_alcampo(data_raw_alcampo[beer_number][3])
    else:
        data_beer_alcampo.iloc[beer_number,3] = 'No promotion'

In [27]:
data_beer_alcampo.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,11.99,1.29,cerveza clásica mahou pack de 28 latas de 33 cl.,oferta válida hasta el 14/10/2020,clásica mahou pack de 28 latas de 33 cl.,lata,33 cl,28,https://www.alcampo.es/media/ha9/h39/100335357...,Alcampo
1,13.49,1.45,cervezas mahou 5 estrellas pack 28 latas x 33 cl.,No promotion,mahou 5 estrellas pack 28 latas x 33 cl.,lata,33 cl,28,https://www.alcampo.es/media/h11/h90/902116140...,Alcampo
2,6.36,1.6,cerveza heineken pack de 12 latas de 33 centi...,No promotion,heineken pack de 12 latas de 33 centilitros,lata,33 c,12,https://www.alcampo.es/media/h67/h4f/954684103...,Alcampo
3,4.67,1.17,"cervezas amstel 100 % malta pack 12 uds, de 3...",No promotion,"amstel 100 % malta pack 12 uds, de 33 cl.",No specified,33 cl,12,https://www.alcampo.es/media/hc9/h32/104726094...,Alcampo
4,6.36,1.6,cervezas clásica mahou pack de 12 latas de 33...,No promotion,clásica mahou pack de 12 latas de 33 centili...,lata,33 c,12,https://www.alcampo.es/media/h64/hc7/965932407...,Alcampo


## El corte ingles

In [39]:
list_pages_corteingles = list(range(1,26))
table_pages_corteingles =[]
for number in list_pages_corteingles:
    url_corteingles = f'https://elcorteingles.es/supermercado/bebidas/cervezas/{number}/'
    html = requests.get(url_corteingles, headers=header).text
    soup_corteingles= BeautifulSoup(html, 'lxml')
    table_corteingles = soup_corteingles.find_all('div',{'class':'grid-item'})
    table_pages_corteingles.append(table_corteingles)

In [40]:
len(table_pages_corteingles)

25

In [41]:
len(table_pages_corteingles[24])

24

In [42]:
data_raw_corteingles=[]
for i in range(len(table_pages_corteingles)):
    for j in range(len(table_pages_corteingles[i])):
        rows_corteingles = table_pages_corteingles[i][j].find_all(['div','h3'],{'class': ['prices-price',
                                                                                    'product_tile-description',
                                                                                    'product_tile-image _fade']})
        promotion_corteingles = table_pages_corteingles[i][j].find_all('div', {'class': 'product_tile-offer offer'})
        if len(rows_corteingles) > 3:
            if len(promotion_corteingles) != 0:
                data_raw_corteingles.append(rows_corteingles + promotion_corteingles)
            else:
                data_raw_corteingles.append(rows_corteingles)
                

In [43]:
len(data_raw_corteingles)

454

In [44]:
for beer in data_raw_corteingles: 
    verification = re.split('cervez|cider|cl',beer[3].text)
    if len(verification)==1:
        beer.pop(1)

In [45]:
#brand corte ingles
def get_brand_corteingles(data_text):
    brand_raw = data_text.text
    brand = re.split('cervez',brand_raw)
    return brand[0].lower()

In [46]:
#volumen corte ingles
def get_volumen_corteingles(data_text):
    brand_raw = data_text.text
    brand = re.split('botel',brand_raw)
    return brand[0]

In [47]:
# #image corte ingles
def get_image_corteingles(data_text):
    return 'https:'+data_text.find_all('img')[0].get('src')

In [48]:
data_raw_corteingles[321][3].text

'ARRIACA cerveza rubia de Castilla la Mancha lata 33 cl '

In [49]:
data_beer_corteingles = pd.DataFrame(index=range(0,len(data_raw_corteingles)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_corteingles)):
    data_beer_corteingles.iloc[beer_number,0] = get_price_alcampo(data_raw_corteingles[beer_number][1])
    data_beer_corteingles.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_corteingles[beer_number][2])
    data_beer_corteingles.iloc[beer_number,2] = get_tittle_alcampo(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,4] = get_brand_corteingles(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,5] = get_container(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,7] = get_quantity_pack(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,8] = get_image_corteingles(data_raw_corteingles[beer_number][0])
    data_beer_corteingles.iloc[beer_number,9] = 'El Corte Ingles'
    if len(data_raw_corteingles[beer_number]) > 4:
        data_beer_corteingles.iloc[beer_number,3] = get_promotion_alcampo(data_raw_corteingles[beer_number][4])
        if len(data_raw_corteingles[beer_number]) > 5:
            data_beer_corteingles.iloc[beer_number,3] += '..SECOND OFFERT: '+ get_promotion_alcampo(data_raw_corteingles[beer_number][5])
    else:
        data_beer_corteingles.iloc[beer_number,3] = 'No promotion'    

In [50]:
data_beer_corteingles.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,5,1.68,ambar cerveza rubia especial pack 9 latas 33 cl,No promotion,ambar,lata,33 cl,9,https://sgfm.elcorteingles.es/SGFM/dctm/MEDIA0...,El Corte Ingles
1,8.15,2.06,voll damm cerveza rubia nacional doble malta p...,No promotion,voll damm,lata,33 cl,12,https://sgfm.elcorteingles.es/SGFM/dctm/MEDIA0...,El Corte Ingles
2,6.72,1.7,mahou clasica cerveza rubia original pack 12 l...,2ª-50%2ª unidad al 50% de descuentoPuedes comb...,mahou clasica,lata,33 cl,12,https://sgfm.elcorteingles.es/SGFM/dctm/MEDIA0...,El Corte Ingles
3,14.84,1.61,mahou clasica cerveza rubia original pack 28 l...,No promotion,mahou clasica,lata,33 cl,28,https://sgfm.elcorteingles.es/SGFM/dctm/MEDIA0...,El Corte Ingles
4,5,1.67,mahou clasica cerveza rubia original pack 12 b...,No promotion,mahou clasica,botella,25 cl,12,https://sgfm.elcorteingles.es/SGFM/dctm/MEDIA0...,El Corte Ingles


## DIA

In [51]:
list_pages_dia= list(range(0,3))
table_pages_dia =[]
for number in list_pages_dia:
    url_dia = f'https://www.dia.es/compra-online/productos/bebidas/cervezas/c/WEB.008.064.00000?page={number}&disp='
    html = requests.get(url_dia, headers=header).text
    soup_dia= BeautifulSoup(html, 'lxml')
    table_dia = soup_dia.find_all('div',{'class':'prod_grid'})
    table_pages_dia.append(table_dia)

In [52]:
len(table_pages_dia)

3

In [53]:
len(table_pages_dia[2])

20

In [54]:
data_raw_dia=[]
for i in range(len(table_pages_dia)):
    for j in range(len(table_pages_dia[i])):
        rows_dia = table_pages_dia[i][j].find_all(['div','span'],{'class': ['price_container','details',
                                                                            'thumb']})
        if len(rows_dia) > 2:
            data_raw_dia.append(rows_dia)
                

In [55]:
len(data_raw_dia)

67

In [56]:
#title Dia
def get_tittle_dia(data_text):
    tittle_raw = data_text.text
    tittle = re.sub('\r\n\t\t\t\t','',tittle_raw)
    return tittle

In [57]:
#brand Dia
def get_brand_dia(data_text):
    brand_raw = data_text.text
    brand_sub = re.sub('\r\n\t\t\t\t','',brand_raw)
    brand = re.split(' cervez',brand_sub)
    return brand[0].lower()

In [58]:
#image dia
def get_image_dia(data_text):
    return data_text.find_all('img')[0].get('data-original')

In [59]:
data_raw_dia[24][2].text

'\n\r\n\t\t\t\t\t\r\n\t\t\t\t\t3,99\xa0€\n(2,66\xa0€/l.)\n'

In [60]:
data_beer_dia= pd.DataFrame(index=range(0,len(data_raw_dia)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_dia)):
    data_beer_dia.iloc[beer_number,0] = get_price_alcampo(data_raw_dia[beer_number][2])
    data_beer_dia.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_dia[beer_number][2])
    data_beer_dia.iloc[beer_number,2] = get_tittle_dia(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,4] = get_brand_dia(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,5] = get_container(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,7] = get_quantity_pack(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,8] = get_image_dia(data_raw_dia[beer_number][0])
    data_beer_dia.iloc[beer_number,9] = 'Dia'
    if len(data_raw_dia[beer_number]) > 3:
        data_beer_dia.iloc[beer_number,3] = get_promotion_alcampo(data_raw_dia[beer_number][3])
    else:
        data_beer_dia.iloc[beer_number,3] = 'No promotion' 

In [61]:
data_beer_dia.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,7.08,1.79,MAHOU 5 ESTRELLAS cerveza pack 12 latas 33 cl,No promotion,mahou 5 estrellas,lata,33 cl,12,https://s3.dia.es/medias/heb/h09/9847113449502...,Dia
1,0.65,1.97,MAHOU MIXTA cerveza con limón lata 33 cl,No promotion,mahou mixta,lata,33 cl,1,https://s0.dia.es/medias/h49/he8/9847069933598...,Dia
2,2.2,1.47,MAHOU cerveza sin alcohol pack 6 botellas 25 cl,No promotion,mahou,botella,25 cl,6,https://s3.dia.es/medias/h25/h47/9847978065950...,Dia
3,0.7,0.7,RAMBLERS cerveza lager botella 1 lt,No promotion,ramblers,botella,1 l,1,https://s0.dia.es/medias/h23/h3e/1023867263388...,Dia
4,0.43,1.3,AMSTEL cerveza lata 33 cl,No promotion,amstel,lata,33 cl,1,https://s3.dia.es/medias/ha3/hd0/9847080747038...,Dia


## Eroski

In [62]:
os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}'  
os.environ['PATH']

driver_options = Options()
driver_options.headless = True
driver = webdriver.Firefox(options=driver_options)

driver.get('https://supermercado.eroski.es/es/supermercado/2060211-bebidas/2060233-cervezas/') 
for i in range(20):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1.1)
    
selenium_raw_eroski = driver.find_elements_by_class_name('product-item')
images = driver.find_elements_by_tag_name('img')

In [63]:
len(selenium_raw_eroski)

276

In [64]:
print(images[200].get_attribute('src'))

https://supermercado.eroski.es/assets/meta/2.35.0.10/imgs/header/img-placeholder.png


In [65]:
list_images=[]
for i in images:
    list_images.append(i.get_attribute('src'))

In [66]:
beer_images_eroski=[]
for i in list_images:
    if 'https://supermercado.eroski.es/images/' in i:
        beer_images_eroski.append(i)


In [67]:
data_raw_eroski = []
for i in selenium_raw_eroski:
    data_raw_eroski.append(i.text)


In [68]:
#price eroski

def get_price_eroski(data_text):
    price = re.findall('[0-9,]+',data_text)
    last = len(price)-1
    return float(re.sub(",",'.',price[last]))

In [69]:
#price_liter eroski

def get_price_liter_eroski(data_text):
    price_raw = re.split(' LITRO A ',data_text)
    if len(price_raw)>1:
        price = re.findall('[0-9,]+',price_raw[1])
    else:
        price = re.findall('[0-9,]{2,}',data_text)
    return float(re.sub(",",'.',price[0]))

In [70]:
#title eroski
def get_tittle_eroski(data_text):
    tittle = re.split('\n',data_text)
    return tittle[1]

In [71]:
#brand eroski
def get_brand_eroski(data_text):
    brand_raw = re.split('erveza |, |ider |idra ',data_text)
    return brand_raw[1].lower()

In [72]:
#container eroski
def get_container_eroski(data_text):
    container = re.findall('botella|lata|barril',data_text)
    if len(container)!=0:
        return container[0]
    else:
        return 'No specified'

In [73]:
#volumen eroski
def get_volumen_unid_eroski(data_text):
    volumen = re.findall('[0-9.,]+ cl',data_text)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9.,]+[ mclL]+',data_text)
        if len(volumen_l)==2:
            return volumen_l[1]
        else:
            return volumen_l[0]

In [74]:
#quantity eroski
def get_quantity_pack_eroski(data_text):
    quantity_split = re.split('pack de|pack',data_text)
    if len(quantity_split)==2:
        quantity = re.findall('[0-9]+',quantity_split[1])
        return quantity[0]
    else:
        return 1

In [75]:
#promotion eroski

def get_promotion_eroski(data_text):
        promotion = re.split("\n",data_text)
        if len(promotion)>8:
            number = len(promotion)-6
            return promotion[number] + ' ' + promotion[number+1]
        else:
            return 'No promotion'


In [76]:
#image eroski

def get_image_eroski(number):
    return beer_images_eroski[number*2]

In [77]:
data_raw_eroski[198]

'  0\nCerveza VOLL-DAMM, botellín 66 cl\n1 LITRO A 2,42 €\nAhora\n1,60 €\nAñadir\nAñadir a la lista'

In [78]:
data_raw_eroski[3]

'  0\nCerveza MAHOU 5 Estrellas, pack 24x33 cl\n1 LITRO A 1,73 €\nAhora\n13,59 €\nAñadir\nAñadir a la lista'

In [79]:
data_beer_eroski= pd.DataFrame(index=range(0,len(data_raw_eroski)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_eroski)):
    data_beer_eroski.iloc[beer_number,0] = get_price_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,1] = get_price_liter_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,2] = get_tittle_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,3] = get_promotion_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,4] = get_brand_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,5] = get_container_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,6] = get_volumen_unid_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,7] = get_quantity_pack_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,8] = get_image_eroski(beer_number)
    data_beer_eroski.iloc[beer_number,9] = 'Eroski'

In [80]:
data_beer_eroski.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,12.59,1.58,"Cerveza SAN MIGUEL, pack lata 24x33 cl",2ª unidad -70%,san miguel,lata,33 cl,24,https://supermercado.eroski.es/images/16514556...,Eroski
1,6.47,1.64,"Cerveza SAN MIGUEL, pack lata 12x33 cl",No promotion,san miguel,lata,33 cl,12,https://supermercado.eroski.es/images/8703969.jpg,Eroski
2,6.72,1.7,"Cerveza MAHOU Clásica, pack 12x33 cl",No promotion,mahou clásica,No specified,33 cl,12,https://supermercado.eroski.es/images/8703977.jpg,Eroski
3,13.59,1.73,"Cerveza MAHOU 5 Estrellas, pack 24x33 cl",No promotion,mahou 5 estrellas,No specified,33 cl,24,https://supermercado.eroski.es/images/16514549...,Eroski
4,5.31,1.76,"Cerveza SAN MIGUEL, pack botellín 12x25 cl",No promotion,san miguel,No specified,25 cl,12,https://supermercado.eroski.es/images/142380.jpg,Eroski


In [29]:
def save_df_products(data,name):
    data.to_csv(f'../data/processed/data_beer_{name}.csv', index=False)

In [87]:
save_df_products(data_beer_carrefour, 'carrefour')
save_df_products(data_beer_alcampo, 'alcampo')
save_df_products(data_beer_corteingles, 'corteingles')
save_df_products(data_beer_dia,'dia')
save_df_products(data_beer_eroski,'eroski')

In [3]:
def load_df_products(name):
    return pd.read_csv(f'../data/processed/data_beer_{name}.csv')

data_beer_carrefour=load_df_products('carrefour')
data_beer_alcampo=load_df_products('alcampo')
data_beer_corteingles=load_df_products('corteingles')
data_beer_dia=load_df_products('dia')
data_beer_eroski=load_df_products('eroski')

In [4]:
data_beer = pd.concat([data_beer_carrefour, 
                       data_beer_alcampo, 
                       data_beer_corteingles, 
                       data_beer_dia, 
                       data_beer_eroski], axis=0, ignore_index=True)

In [5]:
data_beer.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,11.21,1.89,Cerveza Heineken Lager pack de 18 latas de 33 cl.,No promotion,heineken lager,lata,33 cl,18,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
1,14.80,1.60,Cerveza Cruzcampo Pilsen pack de 28 latas de 3...,No promotion,cruzcampo pilsen,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
2,5.15,1.56,Cerveza El Aguila pack de 10 latas de 33 cl.,No promotion,el aguila,lata,33 cl,10,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
3,0.25,0.76,Cerveza Carrefour Holandesa lata 33 cl.,No promotion,carrefour holandesa,lata,33 cl,1,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
4,14.15,1.53,Cerveza Mahou Clásica pack de 28 latas de 33 cl.,No promotion,mahou clásica,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour


In [6]:
data_beer['id'] = data_beer.index

In [7]:
data_beer.shape

(1575, 11)

In [93]:
save_df_products(data_beer, 'total')

In [17]:
#download the beer images
def download_image(url):
    myfile = requests.get(url[1],headers=header)
    open(f'../beer_images/{url[0]}', 'wb').write(myfile.content)

In [18]:
data_beer[['id','image_url']].apply(download_image, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1570    None
1571    None
1572    None
1573    None
1574    None
Length: 1575, dtype: object

In [8]:
def format_image(number):
    image = Image.open(f'../beer_images/{number}').convert('RGB')
    new_image = image.resize((280, 280))
    new_image.save(f'../beer_images/{number}.jpg')

In [9]:
data_beer['id'].apply(format_image)

0       None
1       None
2       None
3       None
4       None
        ... 
1570    None
1571    None
1572    None
1573    None
1574    None
Name: id, Length: 1575, dtype: object